# from C++ to python

see https://pybind11.readthedocs.io/en/stable/basics.html

First, get your python environment ready.
We'll use:
```
jupyter-lab pybind11 numpy matplotlib 
```

Pybind11 is a header-only library, so we don't have to compile anything to install it. Just provide the include path to the compiler.
You can test that everything is working with the following command, that retrieves the compilation flag that we will use later. The output depends on your particular software configuration. The first command will show the include path to use pybind11. The second one will tell us the suffix that we have to use for our compiled C++ library. Check that the python version is correctly detected, and the outputs refer to the same python version. If not, your environment is broken. Closing the current terminal and opening a new one may help.

In [1]:
!python -m pybind11 --includes
print("")
!python3-config --extension-suffix

-I/Users/samuele/opt/anaconda3/envs/pybind_env/include/python3.10 -I/Users/samuele/opt/anaconda3/envs/pybind_env/lib/python3.10/site-packages/pybind11/include

.cpython-310-darwin.so


We can see that the first is the path to python while the second one is the path to `pybind` library. If we go to the second one we can see the C libraries.

# Original C++ code

This comes from the exercise about the toy matrix class (lecture 4). We will implement the python interface of that class in python. Note that I modified the `main` function that is present in the repository, renaming it and removing its arguments. Try to use your own implementation.\
We do not want a main function as it is a library not an executable, so we remove it.

```c++
#include <iostream>
#include <vector>
#include <fstream>



template <typename T>
class CMatrix {
  public:
    int size;
    std::vector<T> data;
    CMatrix(int N){
      size = N;
      data.resize(N*N);
    }
    CMatrix(){};
    void print_to_file(const std::string& file);
    void read_from_file(const std::string& file);
    CMatrix<T> operator*(const CMatrix<T>& B);
   
};


template <typename T>
void CMatrix<T>::read_from_file(const std::string& file){
  std::ifstream filevar(file);
  if(filevar){
      filevar>>size;
      data.resize(size*size);
  for (int i=0;i<size;++i) {
    for (int j=0;j<size;++j) {
      filevar>>data[i*size+j];
    }
  }
  filevar.close();}
    else{
    std::cout<<"coudn't open the file "<<file<<std::endl;    
  }
};



template <typename T>
void CMatrix<T>::print_to_file(const std::string& file){
  std::ofstream filevar(file);
  filevar<<size<<std::endl;
  for (int i=0;i<size;++i) {
    for (int j=0;j<size;++j) {
      filevar<<data[i*size+j]<<" ";
    }
    filevar<<std::endl;
  }
  filevar.close();
};

template <typename T>
CMatrix<T> CMatrix<T>::operator*(const CMatrix<T>& B){
  if (size != B.size) {
    std::cout<<"The two matrices are not of the same size! The result will be nonsense."<<std::endl;
  }
    CMatrix<T> C(size);
    for (int i=0;i<size;i++){
		for (int j=0;j<size;j++){
			for (int k=0;k<size;k++){
				C.data[i*size + j]+=data[i*size + k]*B.data[k*size + j];
			}
		}
	}
	return C;
};



int old_main (){
  CMatrix<double> A,B;
  A.read_from_file("A.txt");
  B.read_from_file("B.txt");
  auto C=A*B;
  C.print_to_file("C.txt");
  
  return 0;
}

```

# Testing a simple function call without arguments
You just have to include your C++ file and the pybind library, which the compiler knows where to find.\
We need to have an hpp file containing our class and functions, and also a cpp file as described below, which is what we want to compile.

matrix_cpp.cpp

```c++
#include "matrix_cpp.hpp"
#include <pybind11/pybind11.h>

PYBIND11_MODULE(matrix_cpp, m) {
    m.doc() = "This module uses the c++ code to perform simple matrix multiplications"; // optional module docstring

    m.def("test_func", &old_main, "execute old main code");

}

```

now we compile the library (adapt the command to your C++ compiler)\
We compile this code as a shared library. Optimizations are important in this case so always turn `-O3` optimization on to ensure we have good binary code.\
`-shared` flag tells the compiler we want to compile as a shared library.\
`-fPIC` Technical stuff for shared libraries, needed to build a library for python.\
Then we substitute the output of this command, then we have the file we want to compile and the output which is a matrix_cpp followed by lst part of the name which depends on the python version. This is essential as it depends on python version and your computer.\
On macOS compile with the additional flag `-undefined dynamic_lookup `

In [2]:
!c++ -O3 -Wall -shared -std=c++17 -fPIC $(python3 -m pybind11 --includes) matrix_cpp.cpp -o matrix_cpp$(python3-config --extension-suffix) -undefined dynamic_lookup 

ld: warning: -undefined dynamic_lookup may not work with chained fixups


Note: on my system `import matrix_cpp` will look for `matrix_cpp.cpython-310-x86_64-linux-gnu.so`. The name of the file depends on the python version to avoid conflicts.\
On Macos it will look for a different one with `.cpython-310-darwin.so` which is in your own directory

In [3]:
import matrix_cpp

In [5]:
matrix_cpp? #docstring returns what we put in the doc module

SyntaxError: invalid syntax (320365086.py, line 1)

Here we write two matrices A.txt and B.txt which will be used in the python code.\
`test_func` takes the address of the old main function and executes it, so it reads the two matrices and performs matrix multiplication. In the end it returns 0 as we specified it that way.

In [6]:
with open('A.txt','w') as f:
    f.write('''3
1 2 3
2 3 4
6 7 8''')
with open('B.txt','w') as f:
    f.write('''3
2 0 0
0 2 0
1 0 2''')

In [7]:
matrix_cpp.test_func?

Docstring:
test_func() -> int

execute old main code
Type:      builtin_function_or_method


In [8]:
matrix_cpp.test_func()

0

We can see that not we have the matrix which is the results of the matrix multiplication of our two files.

In [30]:
with open('C.txt','r') as f:
    print(f.read())

3
5 4 6 
8 6 8 
20 14 16 



# Using a C++ class from python
Now we want to be able to call classes inside python but using c++ class.\

We add a new class by using the `pybind::class_` syntax which takes as argument the C++ class that you want to export as template (`pybind11::class_<DCMatrix >`) and takes as arguments the module m and then you type the name that you will use in python for the class.\
You have to define the initializer class `.def(pybind11::init<>())`, which calls the original constructor. *If constructor has some arguments you need to put them in the template parameter of your class*.\
Then you redefine your function where you specify name of the function and a pointer to a member function of your class **for all member functions** that we had.\
We also need to use this syntax for the assignment operator which is different than the equal operator in C++, so if we want to have this.

matrix_cpp.cpp

```c++
#include "matrix_cpp.hpp"
#include <pybind11/pybind11.h>

PYBIND11_MODULE(matrix_cpp, m) {
    m.doc() = "This module uses the c++ code to perform simple matrix multiplications"; // optional module docstring

    m.def("test_func", &old_main, "execute old main code");
    using DCMatrix = CMatrix<double>;
    pybind11::class_<DCMatrix > (m, "CMatrix")
            .def(pybind11::init<>())
            .def("read_from_file",&DCMatrix::read_from_file)
            .def("print_to_file",&DCMatrix::print_to_file)
            .def("multiply",&DCMatrix::operator*)
            .def("assign",
                           static_cast< DCMatrix&(DCMatrix::*)( const DCMatrix &) > // select the assignment operator
                            (&DCMatrix::operator=)
                            )
            ;

}

```

Better restart the python kernel after compiling the library...

In [9]:
A=matrix_cpp.CMatrix()
B=matrix_cpp.CMatrix()

In [10]:
A.read_from_file?

Docstring: read_from_file(self: matrix_cpp.CMatrix, arg0: str) -> None
Type:      method


In [11]:
A.read_from_file("A.txt")
B.read_from_file("B.txt")

In [12]:
C=A.multiply(B)

In [13]:
A

In [14]:
B

In [15]:
C

In [16]:
C.print_to_file("C_python.txt")

In [17]:
with open('C_python.txt','r') as f:
    print(f.read())

3
5 4 6 
8 6 8 
20 14 16 



In [18]:
D=matrix_cpp.CMatrix()

In [19]:
D.assign(C)

We need to call assign cause the equal operator does a different thing in C++.

In [20]:
D.print_to_file("D_python.txt")

In [21]:
with open('D_python.txt','r') as f:
    print(f.read())

3
5 4 6 
8 6 8 
20 14 16 



# Passing data from C++ to python


buffer protocol interface: the standard that python has to make pieces of python library to communicate with each other.

https://pybind11.readthedocs.io/en/stable/advanced/pycpp/numpy.html

First from C++ to python (easy)\
We want to define a `get_data` function as a lambda function which captures nothing and takes as input a matrix.\
 - First I do some checks (e.g. if size of matrix is 0)
 - We want to give python some data.
     - We allocate the data
     - We copy the original data in this new piece of memory
     - When python garbage collector kicks in we want it to call the destructor which we write inside `free_when_done` otherwise we have memory leak.
 - In python we want to return the data, the shape, the stride, the address of the data and the object that the garbage collector will use.\

*Python and c++ have a completely different way of managing memory, python has a garbage collector which sometimes clearns memory.*

```c++
#include "matrix_cpp.hpp"
#include <pybind11/pybind11.h>
#include <pybind11/numpy.h>
#include <stdexcept>
#include <algorithm>

PYBIND11_MODULE(matrix_cpp, m) {
    m.doc() = "This module uses the c++ code to perform simple matrix multiplications"; // optional module docstring

    m.def("test_func", &old_main, "execute old main code");
    using DCMatrix = CMatrix<double>;
    pybind11::class_<DCMatrix > (m, "CMatrix")
            .def(pybind11::init<>())
            .def("read_from_file",&DCMatrix::read_from_file)
            .def("print_to_file",&DCMatrix::print_to_file)
            .def("multiply",&DCMatrix::operator*)
            .def("assign",
                           static_cast< DCMatrix&(DCMatrix::*)( const DCMatrix &) >
                            (&DCMatrix::operator=)
                            )
            .def("get_data", [](const DCMatrix & d)
                {
                   //allocate copy of matrix data and pass it to python domain
                   //Python decides when to destroy the object.
                   //The data is completely given to the python world
                   
                   if (d.size <=0 ) {
                      throw std::runtime_error("matrix is empty");
                   }
                   auto * python_data = new double[d.size*d.size];
                   std::copy(d.data.begin(),d.data.end(),python_data);

                   //little routine that is called when
                   // the object is collected by the garbage collector
                   pybind11::capsule free_when_done(python_data, [] (void * pointer) {
                       std::cout << "freeing memory @ " << pointer <<std::endl;
                       delete [] static_cast<double*>(pointer);
                   });

                   return pybind11::array_t<double>( // array_t is in pybind11/numpy.h
                       {{d.size,d.size}},//shape
                       {d.size*sizeof(double),sizeof(double)}, //stride
                       python_data,
                       free_when_done
                   );

                }
               )
            ;

}
```


In [46]:
data=C.get_data()

In [47]:
data

array([[ 5.,  4.,  6.],
       [ 8.,  6.,  8.],
       [20., 14., 16.]])

# Passing data from python to C++

The direction python -> C++ is difficult because python buffer protocol supports a lot of memory layouts, to make the computation more efficient. Since we don't want to reimplement all possible layout for our simple C++ toy code, we check that the data that python passes is a simple contiguous array.

We define a new function `set_data` and use another lambda function which takes as input a matrix (equivalent to self) and a python buffer.\
First we get the buffer information using `pybind11::buffer_info info`, then we do some checks (i.e. to ensure we are getting double from the python side) and also have to check for shape (square, well defined, ...).

There is a **stride issue**. You can have an array stored in memory in many different ways, now we just check that the stride we get from python side is a continuous array.

Then if all these checks are good we can copy the data.

```c++
#include "matrix_cpp.hpp"
#include <pybind11/pybind11.h>
#include <pybind11/numpy.h>
#include <stdexcept>
#include <algorithm> 

PYBIND11_MODULE(matrix_cpp, m) {
    m.doc() = "This module uses the c++ code to perform simple matrix multiplications"; // optional module docstring

    m.def("test_func", &old_main, "execute old main code");
    using DCMatrix = CMatrix<double>;
    pybind11::class_<DCMatrix > (m, "CMatrix")
            .def(pybind11::init<>())
            .def("read_from_file",&DCMatrix::read_from_file)
            .def("print_to_file",&DCMatrix::print_to_file)
            .def("multiply",&DCMatrix::operator*)
            .def("assign",
                           static_cast< DCMatrix&(DCMatrix::*)( const DCMatrix &) >
                            (&DCMatrix::operator=)
                            )
            .def("get_data", [](const DCMatrix & d)
                {
                   //allocate copy of matrix data and pass it to python domain
                   //Python decides when to destroy the object.
                   //The data is completely given to the python world

                   if (d.size <=0 ) {
                      throw std::runtime_error("matrix is empty"); 
                   }
                   auto * python_data = new double[d.size*d.size]; 
                   std::copy(d.data.begin(),d.data.end(),python_data);


                   //little routine that is called when
                   // the object is collected by the garbage collector
                   pybind11::capsule free_when_done(python_data, [] (void * pointer) {
                       std::cout << "freeing memory @ " << pointer <<std::endl;
                       delete [] static_cast<double*>(pointer);
                   });

                   return pybind11::array_t<double>( // array_t is in pybind11/numpy.h
                       {{d.size,d.size}},//shape
                       {d.size*sizeof(double),sizeof(double)}, //stride
                       python_data,
                       free_when_done
                   );

                }
               )
            .def("set_data",[](DCMatrix &d, const pybind11::buffer numpy_matrix)
                {  
                   //get info of python array
                   pybind11::buffer_info info{numpy_matrix.request()};
                   //check that we are dealing with an array of double
                   if (info.format != pybind11::format_descriptor<double>::format()) {
                      throw std::runtime_error("we can accept only a matrix made with C double type");
                   }
                   //sanity check
                   if (info.ndim != 2) {
                       throw std::runtime_error("dimension of array must be 2");
                   }
                   if (info.shape[0] != info.shape[1]){
                       throw std::runtime_error("we implemented only square matrices, sorry");
                   }
                   if (info.shape[0]<=0) {
                      throw std::runtime_error("dimension of the matrix is zero");
                   }
                   // to simplify the logic, implement only contiguous arrays. Check that the array is contiguous
                   ssize_t stride=sizeof(double);
                   for (int i=info.ndim-1;i>=0;--i){
                      if (info.strides[i] != stride) {
                         throw std::runtime_error("sorry, we don't support a non-contiguous matrix");
                      }
                      stride *= info.shape[i];
                   }

                   //all sanity checks are passed, copy the data
                   d = DCMatrix(info.shape[0]); //use assignment to not write other code...
                   std::copy(static_cast<double*>(info.ptr),static_cast<double*>(info.ptr)+info.shape[0]*info.shape[1],d.data.begin());
                   //c++ is responsable of this data (managed by the vector class)
                }
               )
            ;

}

```

In [48]:
import numpy as np

Here for example if we try to change `dtype` to int you will get an error as you have a double matrix.\
pybind11 automatically converts all the python errors into c++ errors.

In [49]:
C.set_data(np.array([[1,2,3],[2,3,4],[1,2,3]],dtype=float))

In [50]:
C.get_data()

array([[1., 2., 3.],
       [2., 3., 4.],
       [1., 2., 3.]])

## optional: clean the produced files

(remove the '#')

In [51]:
!#rm A.txt B.txt C.txt C_python.txt D_python.txt matrix_cpp.*.so

rm: D_python.txt: No such file or directory
